# [백준/세상의 중심에서...](https://www.acmicpc.net/problem/2389)


## 풀이과정


### 첫번째 시도


#### 풀이과정

모노톤 체인을 이용해 최외각 값들을 구한 뒤 휴리스틱으로 외접원을 구해 해결했다.  
[보양쿠님의 풀이](https://velog.io/@vkdldjvkdnj/boj02389)를 참조했다.


In [ ]:
from itertools import accumulate
from typing import Callable, Iterable


def mean(points: list[complex]) -> complex:
    return sum(points) / len(points)


def ccw(a: complex, b: complex, c: complex):
    return ((a - c).conjugate() * (b - c)).imag > 0


def dist(a: complex) -> Callable[[complex], float]:
    return lambda x: abs(a - x)


def key(a: complex):
    return a.real, a.imag


def disc(x: float, _) -> float:
    return x * 0.995


def get_halved_hull(points: Iterable[complex]) -> list[complex]:
    result: list[complex] = []
    for p in points:
        while len(result) > 1 and not ccw(p, *result[-2:]):
            result.pop()
        result.append(p)
    return result


def monotone_chain(points: list[complex]) -> list[complex]:
    return get_halved_hull(points)[:-1] + get_halved_hull(reversed(points))[:-1]


def heuristic(hull: list[complex]) -> tuple[float, float, float]:
    m = mean(hull)
    for r in accumulate(range(5000), disc, initial=0.1):
        m += (max(hull, key=dist(m)) - m) * r
    return m.real, m.imag, max(map(dist(m), hull))


def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    if len(inputs) == 1:
        return print(*inputs, 0)
    points = sorted((complex(*map(float, line.split())) for line in inputs), key=key)
    print(*heuristic(monotone_chain(points)))


solution()

### 두번째 시도


#### 풀이과정

[thxios457 님](https://www.acmicpc.net/user/thxios457)의 [55045393 번 풀이](https://www.acmicpc.net/source/55045393)를 참고해 풀어봤다.  
먼저 두 점을 잇는 선분을 지름으로 하는 원을 구한 뒤 해당 원을 시작으로 점을 추가해나가며 외접원을 구하는 방식이다.  
해당 알고리즘은 [이 글](https://greimul.tistory.com/33)에서 처음 접하고 이용해 풀려고 했으나 실패했었는데, 55045393 번 풀이가 이 알고리즘을 잘 구현해서 참고했다.


In [ ]:
# refer https://www.acmicpc.net/source/55045393


def cross(a: complex, b: complex) -> float:
    return (a.conjugate() * b).imag


def get_center_2(a: complex, b: complex) -> complex:
    return (a + b) / 2


def get_center_3(a: complex, b: complex, c: complex) -> complex:
    ab = b - a
    ac = c - a
    return a + (abs(ac) ** 2 * ab - abs(ab) ** 2 * ac) * 1j / (2 * cross(ab, ac))


def get_circle(bound: list[complex]):
    center = (get_center_3 if len(bound) > 2 else get_center_2)(*bound)
    return center, abs(center - bound[0])


def init(bound: list[complex]) -> tuple[complex, float]:
    return get_circle(bound) if len(bound) > 1 else ((bound[0] if bound else 0j), 0.0)


def enclosed(bound: list[complex], points: list[complex]) -> tuple[complex, float]:
    if len(bound) == 3:
        return get_circle(bound)
    center, radius = init(bound)
    for i, p in enumerate(points):
        if abs(center - p) > radius:
            center, radius = enclosed(bound + [p], points[:i])
    return center, radius


def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    points = list(set(complex(*map(float, line.split())) for line in inputs))
    center, radius = enclosed([], points)
    print(center.real, center.imag, radius)


solution()

![2024년 1월 15일 14시 기준 2389번 맞힌 사람 (Python) 40ms로 1위](<../../img/Screenshot 2024-01-15 at 14-48-37 2389번 맞힌 사람 (Python) - 1 페이지.png>)


## 해답


In [1]:
def cross(a: complex, b: complex) -> float:
    return (a.conjugate() * b).imag

In [4]:
def get_center_2(a: complex, b: complex) -> complex:
    return (a + b) / 2

In [5]:
def get_center_3(a: complex, b: complex, c: complex) -> complex:
    ab = b - a
    ac = c - a
    return a + (abs(ac) ** 2 * ab - abs(ab) ** 2 * ac) * 1j / (2 * cross(ab, ac))

In [6]:
def get_circle(bound: list[complex]):
    center = (get_center_3 if len(bound) > 2 else get_center_2)(*bound)
    return center, abs(center - bound[0])

In [7]:
def init(bound: list[complex]) -> tuple[complex, float]:
    return get_circle(bound) if len(bound) > 1 else ((bound[0] if bound else 0j), 0.0)

In [8]:
def enclosed(bound: list[complex], points: list[complex]) -> tuple[complex, float]:
    if len(bound) == 3:
        return get_circle(bound)
    center, radius = init(bound)
    for i, p in enumerate(points):
        if abs(center - p) > radius:
            center, radius = enclosed(bound + [p], points[:i])
    return center, radius

In [9]:
def solution():
    import sys

    inputs = sys.stdin.read().strip().split("\n")[1:]
    points = list(set(complex(*map(float, line.split())) for line in inputs))
    center, radius = enclosed([], points)
    print(center.real, center.imag, radius)

## 예제


In [10]:
# 백준 문제 풀이용 예제 실행 코드
from bwj import test

test_solution = test(solution)

# test_solution("""""")
# test_solution(read("fn").read())

In [11]:
test_solution(
    """3
1 1
2 2
3 3
"""
)  # 2 2 1.4142135624

2.0 2.0 1.4142135623730951
